<a href="https://colab.research.google.com/github/abanduc/proyecto-gestion-financiera/blob/main/Hito2_GF_AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Valoración opciones aplicada

Tome en consideración el modelo continuio de Black and Scholes:

$$C(S,t)=S\cdot\Phi(d_1)-K\exp(-R\cdot t)\cdot\Phi(d_2)$$

donde $d_1=\frac{\log\frac{S}{K}+\left(R+\frac{\sigma^2}{2}\right)\cdot t}{\sqrt{\sigma^2\cdot t}}$ y $d_2=d_1-\sqrt{\sigma^2\cdot t}$.

Defina los parámetros asociados a valores del activo definido en la Tarea 1 y calcule el valor de una opción de compra.

In [6]:
!pip install numpy scipy


In [7]:
####################################
#      MODELO BLACK AND SCHOLES
####################################
import numpy as np
from scipy.stats import norm

# Datos del activo y opción
precio_activo = 6458.9
precio_ejercicio = 6461.7

# Datos financieros
tasa_interes = 0.032  # Tasa de interés libre de riesgo en Chile
volatilidad = 0.157  # Volatilidad del activo
tiempo_expiracion = 0.5  # Tiempo hasta la expiración (en años)

# Cálculo de d1 y d2 para Black-Scholes
log_price_ratio = np.log(precio_activo / precio_ejercicio)
volatility_term = volatilidad * np.sqrt(tiempo_expiracion)
d1 = (log_price_ratio + (tasa_interes + (volatilidad ** 2) / 2) * tiempo_expiracion) / volatility_term
d2 = d1 - volatility_term

# Fórmula Black-Scholes para valorizar una opción de compra
call_option_value = precio_activo * norm.cdf(d1) - precio_ejercicio * np.exp(-tasa_interes * tiempo_expiracion) * norm.cdf(d2)

# Resultados
print(f"Usando el modelo Black-Scholes, el valor de la opción de compra es: {call_option_value:.2f}")


Usando el modelo Black-Scholes, el valor de la opción de compra es: 336.37


In [8]:
####################################
#      MODELO MONTECARLO
####################################
num_simulaciones = 10000
precio_activo = 100
tasa_interes = 0.05
volatilidad = 0.2
tiempo_expiracion = 1.0
precio_ejercicio = 105

np.random.seed(0)
simulaciones = np.random.normal(0, 1, num_simulaciones)
descuento = np.exp(-tasa_interes * tiempo_expiracion)


precio_futuro = precio_activo * np.exp((tasa_interes - 0.5 * volatilidad**2) * tiempo_expiracion + volatilidad * np.sqrt(tiempo_expiracion) * simulaciones)
pagos = np.maximum(precio_futuro - precio_ejercicio, 0)

valor_opcion_compra = descuento * np.mean(pagos)

print(f"Valor de la opción de compra (Monte Carlo): {valor_opcion_compra:.2f}")


Valor de la opción de compra (Monte Carlo): 7.70


In [27]:
####################################
#      MODELO BINOMIAL
####################################

import yfinance as yf
import datetime as dt

def binomial_option_pricing(S, K, T, r, sigma, option_type='call', n=98):
   # Calcula el paso de tiempo y el factor de descuento
    dt = T / n
    discount_factor = np.exp(-r * dt)

    # Calcula los valores de movimiento hacia arriba y hacia abajo
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u

    # Calcula las probabilidades neutrales al riesgo
    q = (np.exp(r * dt) - d) / (u - d)

    # Inicializa una matriz para almacenar los valores de la opción en cada nodo del árbol
    option_values = np.zeros((n + 1, n + 1))

    # Llena la matriz de valores de opción en función del modelo de árbol binomial
    for j in range(n + 1):
        if option_type == 'call':
            option_values[n, j] = max(0, S * (u**j) * (d**(n - j)) - K)
        else:
            option_values[n, j] = max(0, K - S * (u**j) * (d**(n - j)))

    for i in range(n - 1, -1, -1):
        for j in range(i + 1):
            if option_type == 'call':
                option_values[i, j] = max(0, (q * option_values[i + 1, j] + (1 - q) * option_values[i + 1, j + 1]) * discount_factor)
            else:
                option_values[i, j] = max(0, (q * option_values[i + 1, j] + (1 - q) * option_values[i + 1, j + 1]) * discount_factor)

    return option_values[0, 0]

symbol =  "COPEC.SN"
asset_data = yf.Ticker(symbol)
precio_accion = asset_data.history(period='1d')['Close'][0]

data = yf.download(symbol, start='2023-01-01', end=dt.date.today().strftime("%Y-%m-%d"))

data["Daily_Returns"] = data["Adj Close"].pct_change()
volatilidad = data["Daily_Returns"].std()

opcion = yf.Ticker("SPG231215C00055000")
precio_strike = opcion.info["strikePrice"]
fecha = dt.datetime.utcfromtimestamp(opcion.info["expireDate"])
tiempo_para_vencimiento = (np.datetime64(fecha) - np.datetime64('today')) / np.timedelta64(1, 'D')

treasury_data = yf.Ticker("^TNX")
treasury_history = treasury_data.history(period="1d")
tasa_libre_de_riesgo = treasury_history['Close'][0] / 100

option_type = 'call'

option_price = binomial_option_pricing(precio_accion, precio_strike, tiempo_para_vencimiento / 365, tasa_libre_de_riesgo, volatilidad, option_type)

print(f"Precio de la opción {option_type}: {option_price:.2f}")



[*********************100%%**********************]  1 of 1 completed
Precio de la opción call: 6164.12


##Reporte Grupal: Caso Administración


Se eligen uno de los siguientes casos con tal de replicar y responder sus preguntas de acuerdo al activo principal elegido:

Caso Administración de Capital de Trabajo de Keafer Manufacturing de la página 822 del ROSS.

Caso Administración de Efectivo de Richmond Corporation de la página 845 del ROSS.

Caso Administración de Efectivo de Braam Industries de la página 874 del ROSS.

En este caso, el grupo eligió el Caso Administración de Efectivo de Braam Industries, donde se debe examinar la política de crédito de la compañía para ver si una política diferente podría contribuir a aumentar la rentabilidad. En ese caso, una vez determinada la política de crédito actual, se proponen 3 opciones para cambiar esta política, y se debe determinar cuál es la mejor alternativa y determinar inconsistencias.

Este caso se replicó para la compañía Parque Arauco. Para esto, se buscó en los E.E.F.F. de la compañía para el año 2022 toda la información necesaria para poder resolver el caso, tales como los ingresos por ventas, el monto en cuentas por cobrar, los costos administrativos, los costos variables de la operación, etc. La información encontrada fue la siguiente:


In [9]:
# Definición de los datos
ingresos_ventas_2022 = 243119543000
deudores_2022 = 22420967000
deudores_2021 = 23653332000
costos_administrativos = 22424419000
costo_ventas = 55126662000

# Imprimir la tabla
print("Ingresos por Ventas Anuales (CLP$)\t", f"${ingresos_ventas_2022:,}")
print("Monto Neto de Deudores por venta 2022 (CLP$)\t", f"${deudores_2022:,}")
print("Monto Neto de Deudores por venta 2021 (CLP$)\t", f"${deudores_2021:,}")
print("Costos Administrativos (CLP$)\t\t", f"${costos_administrativos:,}")
print("Costo de las Ventas (CLP$)\t\t\t", f"${costo_ventas:,}")


Ingresos por Ventas Anuales (CLP$)	 $243,119,543,000
Monto Neto de Deudores por venta 2022 (CLP$)	 $22,420,967,000
Monto Neto de Deudores por venta 2021 (CLP$)	 $23,653,332,000
Costos Administrativos (CLP$)		 $22,424,419,000
Costo de las Ventas (CLP$)			 $55,126,662,000


Con esta información, fue posible obtener las ventas anuales, la tasa de incumplimiento y los costos administrativos como porcentaje de las ventas, además del periodo de cuentas por cobrar y los costos variables que se tienen con la política de crédito actual.

La tasa de incumplimiento como porcentaje de las ventas corresponde a la división entre el Monto Neto de Deudores por Venta para el año 2022 y las Ventas Anuales. Los costos administrativos como porcentaje de las ventas corresponden a la división entre los Costos Administrativos y las Ventas Anuales. El periodo de cuentas por cobrar corresponde a la división entre el promedio del Monto Neto de Deudores inicial y final, con las Ventas Anuales, y todo eso dividido por 365 para obtener la cantidad de días. Por último, los costos variables como porcentaje corresponden a la división entre los Costos de las Ventas y las Ventas Anuales.

Los resultados se presentan a continuación:


In [10]:
# Definición de los datos
ventas_anuales = 243119543000
tasa_incumplimiento = 9.222199
costos_administrativos = 9.22362
periodo_cuentas_cobrar = 34.5861
costos_variables = 23

# Imprimir la tabla
print("Ventas Anuales (CLP)\t\t\t", f"${ventas_anuales:,}")
print("Tasa de Incumplimiento (% Ventas)\t", f"{tasa_incumplimiento:.6f}%")
print("Costos Administrativos (% Ventas)\t", f"{costos_administrativos:.5f}%")
print("Periodo de cuentas por cobrar (Días)\t", f"{periodo_cuentas_cobrar:.4f}")
print("Costos Variables\t\t\t", f"{costos_variables}%")


Ventas Anuales (CLP)			 $243,119,543,000
Tasa de Incumplimiento (% Ventas)	 9.222199%
Costos Administrativos (% Ventas)	 9.22362%
Periodo de cuentas por cobrar (Días)	 34.5861
Costos Variables			 23%


También, es importante señalar que para los cálculos que vienen a continuación, se utilizó la tasa de interés promedio de aquel periodo, correspondiente a un 8,375%.

En lo que respecta a las diferentes opciones estas difieren en las ventas anuales, tasa de incumplimiento, costos administrativos y periodo de cuentas por cobrar. Luego, para evaluar cada opción se realiza el cálculo del Valor Presente Neto de cada una. De esta forma, se escoge la opción la cuál obtiene un VPN más alto, en comparación a las demás. Los resultados fueron los que se presentan a continuación.


In [11]:
# Definición de los datos
vpn = {
    1: 43300724413.19,
    2: 38355341091.22,
    3: 43791215884.26
}

# Imprimir la tabla
print("Opción\tVPN (CLP)")
for opcion, valor in vpn.items():
    print(f"{opcion}\t{valor:,.2f}")


Opción	VPN (CLP)
1	43,300,724,413.19
2	38,355,341,091.22
3	43,791,215,884.26


De donde se observa que la opción que obtiene el mayor valor es la opción 3, luego esta debería ser la política a adoptar por parte de la empresa. Ahora, en cuanto a los costos de incumplimiento y costos administrativos de la opción 3 y 2, es extraño que si la opción 3 genera más ingresos, esta tenga menores costos asociados. Por lo tanto, aunque los gastos y las ventas no siempre están directamente relacionados, al examinar los detalles, se observa que los gastos de la opción 3 no son inferiores a los de la opción 2. Por consiguiente, la afirmación inicial no parece factible.

